# Run YOLO detection on one of the trained models

In [ ]:
#@title Parameters {run:"auto"}
version_notes = "No TTA" #@param {type:"string"}

YOLO_VERSION =   43#@param {type:"number"}
YOLO_VERSION_NOTES = "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold3of5_WBFat0.4iou_det0.6iou0.01conf.exp" #@param {type:"string"}

DETECT_IOU = 0.6 #@param {type:"slider", min:0, max:1, step:0.01}
DETECT_CONF = 0.01 #@param {type:"slider", min:0, max:1, step:0.01}

IMAGE_SIZE = 640 #@param {type:"slider", min:512, max:1024, step:64}

In [ ]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/Colab\ Notebooks/kaggle
    from setup_colab import setup_colab_for_kaggle, WORK_FOLDER, dump_dataset_metadata
    setup_colab_for_kaggle(check_env=False, local_working=True)
except:
    print("Not in Colab")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/kaggle
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Content of Drive Kaggle data dir (/content/drive/MyDrive/kaggle): ['/content/drive/MyDrive/kaggle/input', '/content/drive/MyDrive/kaggle/working', '/content/drive/MyDrive/kaggle/.ipynb_checkpoints', '/content/drive/MyDrive/kaggle/output']
Content of Kaggle data dir (/kaggle): ['/kaggle/working', '/kaggle/output', '/kaggle/input']
Content of Kaggle data subdir (/kaggle/input): ['/kaggle/input/cassava-model', '/kaggle/input/cassava-leaf-disease-classification', '/kaggle/input/googlebitemperedloss', '/kaggle/input/vbdyolo', '/kaggle/input/.ipynb_checkpoints', '/kaggle/input/vinbigdata', '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection', '/kaggle/input/vinbigdata-chest-xray-origi

In [ ]:
from IPython.display import clear_output

INPUT_FOLDER_YOLO_OUT = WORK_FOLDER / "vbdyolo-out"
INPUT_FOLDER_PNG = WORK_FOLDER / "vinbigdata"
OUTPUT_FOLDER_CUR = WORK_FOLDER / "vbdyolo-out-detect"

## Get YOLO model and data

In [ ]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@add-datasets-download-version
clear_output()

Get the YOLO model

In [ ]:
!rm -rf {INPUT_FOLDER_YOLO_OUT}
!kaggle datasets download "witalia/vbdyolo-out-newest" -v {YOLO_VERSION} -p {INPUT_FOLDER_YOLO_OUT} --unzip --force

100% 311M/311M [00:08<00:00, 32.7MB/s]
100% 311M/311M [00:08<00:00, 37.9MB/s]


Get the test images

In [ ]:
!kaggle datasets download xhlulu/vinbigdata-chest-xray-resized-png-1024x1024 -p {INPUT_FOLDER_PNG} --unzip

100% 7.34G/7.35G [01:23<00:00, 89.7MB/s]
100% 7.35G/7.35G [01:23<00:00, 94.4MB/s]


## Set up YOLO

In [ ]:
%cd $WORK_FOLDER
!rm -rf yolov5 && git clone https://github.com/ultralytics/yolov5 # && cd yolov5 && git checkout tags/v4.0 && git checkout -b exp
%cd yolov5
%pip install -qr requirements.txt

/kaggle/working
Cloning into 'yolov5'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 5595 (delta 61), reused 70 (delta 28), pack-reused 5476
Receiving objects: 100% (5595/5595), 8.44 MiB | 14.46 MiB/s, done.
Resolving deltas: 100% (3802/3802), done.
Note: checking out 'tags/v4.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 69be8e7 YOLOv5 v4.0 Release (#1837)
/kaggle/working/yolov5


## Run detection with pretrained weights

In [ ]:
EXPERIMENT_NAME = "exp"

In [ ]:
!python detect.py \
    --name {EXPERIMENT_NAME} \
    --weights {INPUT_FOLDER_YOLO_OUT}/yolov5_xray_best.pt \
    --img {IMAGE_SIZE} \
    --conf-thres {DETECT_CONF} \
    --iou-thres {DETECT_IOU} \
    --source {INPUT_FOLDER_PNG}/test \
    --save-txt \
    --save-conf
    # --augment

# Make sure to collect predictions.
!ls runs/detect/{EXPERIMENT_NAME}/labels/ | wc -l
# Make sure to delete any exising labels in the folder, since they will get merged.
!rm -rf {OUTPUT_FOLDER_CUR}/labels_pred/
!mkdir -p {OUTPUT_FOLDER_CUR}/labels_pred/
!cp -r runs/detect/{EXPERIMENT_NAME}/labels/*.txt {OUTPUT_FOLDER_CUR}/labels_pred/

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.01, device='', exist_ok=False, img_size=640, iou_thres=0.6, name='exp', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/working/vinbigdata/test', update=False, view_img=False, weights=['/kaggle/working/vbdyolo-out/yolov5_xray_best.pt'])
Using torch 1.8.0+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)


Fusing layers... 
Model Summary: 476 layers, 87286171 parameters, 0 gradients, 217.4 GFLOPS
image 1/3000 /kaggle/working/vinbigdata/test/002a34c58c5b758217ed1f584ccbcfe9.png: 640x640 1 Aortic enlargements, 1 Cardiomegalys, 4 ILDs, 1 Infiltrations, 1 Lung Opacitys, 1 Other lesions, 12 Pleural thickenings, Done. (0.068s)
image 2/3000 /kaggle/working/vinbigdata/test/004f33259ee4aef671c2b95d54e4be68.png: 640x640 2 Aortic enlargements, Done. (0.048s)
image 3/3000 /kaggle/working/vinbigdata/test/008bdde2af2462e86fd373a445d0f4cd.png: 640x640 2 Aortic enlargements, 1 Cardiomegalys, 6 Pleural thickenin

## Store predictions as another version of the dataset

Copy model files

In [ ]:
!cp {INPUT_FOLDER_YOLO_OUT}/yolov5_xray_best.pt {OUTPUT_FOLDER_CUR}/yolov5_xray_best.pt
!cp {INPUT_FOLDER_YOLO_OUT}/yolov5_xray.pt {OUTPUT_FOLDER_CUR}/yolov5_xray.pt

Store to Kaggle

In [ ]:
version_message = " ++ ".join([
    version_notes,
    YOLO_VERSION_NOTES
])
!echo "{version_message}"
dump_dataset_metadata("witalia", "vbdyolo-out-newest", OUTPUT_FOLDER_CUR)
# !kaggle datasets create -p {OUTPUT_FOLDER_CUR} -r zip
!kaggle datasets version -m "{version_message}" -p {OUTPUT_FOLDER_CUR} -r zip

No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold3of5_WBFat0.4iou_det0.6iou0.01conf.exp
Starting upload for file labels_pred.zip
100% 1.32M/1.32M [00:10<00:00, 135kB/s]
Upload successful: labels_pred.zip (1MB)
Starting upload for file yolov5_xray_best.pt
100% 167M/167M [00:20<00:00, 8.68MB/s]
Upload successful: yolov5_xray_best.pt (167MB)
Starting upload for file yolov5_xray.pt
100% 167M/167M [00:18<00:00, 9.49MB/s]
Upload successful: yolov5_xray.pt (167MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/witalia/vbdyolo-out-newest
